# 第三章：提示词工程 (Prompt Engineering)

> 💡 **核心观点**：即使是相同模型，不同的提示词会产生不同的结果

## 1. 提示词工程概述

### 1.1 定义

**提示词工程（Prompt Engineering）**：通过设计和优化输入语言提示，引导 LLM 或智能体系统按照预期目标生成内容、执行操作的技术和方法。

---

## 2. 提示工程核心概念

### 2.1 提示词的组成

提示词通常由以下两部分组成：

- **系统提示（System Prompt）**
  - 影响每一次对话
  - 设定模型的角色、行为准则和输出格式

- **用户提示（User Prompt）**
  - 影响本次对话
  - 包含具体的任务指令和上下文信息

### 2.2 内容组织原则

- **内容划分 & 结构划分**
  - 关键内容放在**开头和结尾**
  - 利用首因效应和近因效应提高模型关注度

### 2.3 基本原则

设计提示词时应遵循以下基本原则：

1. ✅ **清晰**：表达明确，避免歧义
2. ✅ **具体**：提供详细的指令和要求
3. ✅ **上下文适配**：考虑背景信息和场景
4. ✅ **可验证性**：输出结果可被验证和评估
5. ✅ **适度冗余**：关键信息可适当重复强调

---

## 3. 提示工程常见技术

### 3.1 零样本学习（Zero-Shot Learning）

**角色扮演（Role Playing）**

- **原理**：在提示词中明确指定模型扮演的身份角色或职业
- **效果**：影响模型的语言风格、思考角度和行为模式
- **示例**：
  ```
  你是一位经验丰富的软件架构师，请分析以下代码...
  ```

### 3.2 少样本学习（Few-Shot Learning）

**上下文学习（In-Context Learning）**

- **原理**：在提示词中放入示例（examples）
- **进阶**：可以与思维链（CoT）结合，放入包含推理逻辑的例子
- **优势**：让模型通过示例学习任务模式和期望输出格式

### 3.3 思维链（Chain of Thought, CoT）

- **核心思想**：`Let's think step by step`
- **方法**：引导模型逐步推理，展示思考过程
- **效果**：提高复杂推理任务的准确性和可解释性

### 3.4 自我一致（Self-Consistency）

- **原理**：多次生成答案，选择出现次数最多的答案
- **适用场景**：需要高准确性的任务
- **方法**：通过多次采样，利用"多数投票"机制提高可靠性

### 3.5 自我反思（Self-Reflection）

**相关技术**：

- `self-reflection`
- `ReACT` (Reasoning + Acting)
- `Actor-Evaluator`
- `Self-Reflexion`

**原理**：让模型对自己的输出进行评估和反思，迭代改进

---

## 4. 提示工程调优方法及实践

### 4.1 提示工程调优流程（HumanRF）

#### 核心步骤

1. **拆解任务，指令化表达**
   - 将复杂任务分解为简单步骤
   - 用清晰的指令描述每个步骤

#### 迭代优化流程

```
初版提示词
    ↓
输出结果
    ↓
优化步骤：
  • 分步提示化
  • 加入示例
  • 观察输出偏差
  • 添加约束条件
  • 调整措辞
  • 角色设定优化
    ↓
下一版提示词
```

### 4.2 优化要点

| 优化方向 | 具体方法 |
|---------|---------|
| **结构化** | 分步提示化，明确任务步骤 |
| **示例化** | 加入少样本示例，引导输出格式 |
| **约束化** | 添加约束条件，限制输出范围 |
| **角色化** | 设定合适的角色，影响输出风格 |
| **措辞优化** | 调整表达方式，提高指令清晰度 |

---

## 5. 总结

提示词工程是充分发挥大语言模型能力的关键技术。通过合理设计提示词结构、运用各种技术方法，并持续迭代优化，可以显著提升模型在特定任务上的表现。

**关键要点**：
- 🎯 清晰、具体、可验证的提示词设计
- 🔄 结合零样本、少样本、思维链等技术
- 📈 通过迭代优化不断提升效果
    


    



---

## 6. 实践代码

### 6.1 环境准备

在使用提示工程之前，首先需要准备 API 调用环境。以下是一个通用的 API 调用函数：

In [6]:
# 从utilities模块导入API配置加载和get_response函数
import sys
import os
from pathlib import Path

# 方法1: 如果Notebook在HandsOn目录下，直接导入utilities
try:
    from utilities.api_config import load_config, get_response
    print("✅ 使用相对导入成功加载 utilities 模块")
except ImportError:
    # 方法2: 如果相对导入失败，添加项目根目录到Python路径
    current_dir = Path.cwd()
    
    # 如果当前在HandsOn目录，项目根目录是上一级
    if current_dir.name == "HandsOn":
        project_root = current_dir.parent
    elif (current_dir / "HandsOn").exists():
        project_root = current_dir
    else:
        # 尝试向上查找包含HandsOn目录的父目录
        project_root = current_dir
        while project_root != project_root.parent:
            if (project_root / "HandsOn").exists():
                break
            project_root = project_root.parent
    
    # 添加项目根目录到Python路径
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    
    # 现在尝试从HandsOn.utilities导入
    from HandsOn.utilities.api_config import load_config, get_response
    print(f"✅ 使用绝对导入成功加载 utilities 模块")
    print(f"   项目根目录: {project_root}")

# 加载API配置（get_response函数会自动使用配置）
config = load_config()

# get_response函数已经定义在api_config模块中，可以直接使用
# 函数签名: get_response(system_prompt="", user_prompt="", model=None, 
#                        temperature=0.0, top_p=1.0, max_tokens=2048, 
#                        platform="openai", config=None, agent=None)

print("✅ API配置和函数初始化完成")
print("   get_response函数已可用，可以直接调用")
print("   示例: response = get_response(system_prompt='你是一个助手', user_prompt='你好')")


✅ 使用绝对导入成功加载 utilities 模块
   项目根目录: c:\Users\Peiyan Li\Desktop\Hands-on-aai
✅ 成功加载配置文件: c:\Users\Peiyan Li\Desktop\Hands-on-aai\HandsOn\config.json
✅ API配置和函数初始化完成
   get_response函数已可用，可以直接调用
   示例: response = get_response(system_prompt='你是一个助手', user_prompt='你好')


---

### 6.2 系统提示与用户提示实践

让我们通过实际示例来理解系统提示和用户提示的区别和作用：

In [7]:
# 示例 1: 基础系统提示和用户提示
system_prompt = "你是一名助教，需要温柔的回答同学们的问题。"  # 系统提示
user_prompt = "在做提示工程时，什么内容适合放入系统提示、什么内容适合放入用户提示？" 

response = get_response(system_prompt, user_prompt)
print(response)

✅ 成功加载配置文件: c:\Users\Peiyan Li\Desktop\Hands-on-aai\HandsOn\config.json
✅ 智能体初始化成功
   平台: OPENAI
   模型: gpt-4o-mini
   API地址: https://api.openai.com/v1
在提示工程中，系统提示和用户提示的内容有不同的侧重点。

**系统提示**通常用于设定模型的行为和语境，适合放入以下内容：
1. **角色设定**：明确模型的身份，比如“你是一位友好的助教”。
2. **任务指令**：指示模型需要完成的具体任务，比如“请回答以下问题”。
3. **风格和语气**：设定回答的风格，比如“请用温柔的语气回答”。
4. **上下文信息**：提供必要的背景信息，以帮助模型更好地理解问题。

**用户提示**则是用户直接输入的内容，适合放入以下内容：
1. **具体问题**：用户想要了解的具体内容，比如“请解释一下提示工程的概念”。
2. **请求信息**：用户希望获取的特定信息，比如“给我一些关于系统提示的例子”。
3. **反馈和指令**：用户对模型回答的反馈或进一步的指令，比如“请更详细地解释”或“用简单的语言再说一遍”。

希望这些信息能帮助你更好地理解如何组织提示内容！如果还有其他问题，随时问我哦！


可以看到，模型以温柔的语气回答了问题。现在让我们在系统提示中加入格式要求，看看模型如何按照指定格式输出：

In [8]:
# 示例 2: 带格式要求的系统提示
system_prompt = """
你是一名助教，需要温柔的回答同学们的问题。
你回答问题时必须先对问题进行充分的解释，再通过举例支持和形象化你的解释，最后对问题做出一句话的总结。
你的回答必须遵循以下格式：
【解释】：
【示例】：
【总结】：
"""  
user_prompt = "在做提示工程时，什么内容适合放入系统提示、什么内容适合放入用户提示？" 

response = get_response(system_prompt, user_prompt)
print(response)

【解释】：在提示工程中，系统提示和用户提示的内容有着不同的功能和目的。系统提示通常是由系统提供的，旨在引导用户如何使用系统或提供必要的背景信息。这些提示通常是固定的，帮助用户理解系统的功能和操作。而用户提示则是用户自己输入的内容，通常是为了引导系统生成特定的响应或结果。用户提示可以是问题、请求或指令，旨在引导系统的行为。

【示例】：例如，在一个聊天机器人应用中，系统提示可能是“欢迎使用我们的服务！请问有什么我可以帮助您的吗？”这条提示是为了让用户知道他们可以开始与系统互动。而用户提示可能是“请告诉我今天的天气如何？”这条提示是用户向系统发出的请求，期望得到具体的信息。

【总结】：系统提示用于引导用户使用系统，而用户提示则是用户与系统互动的具体请求。


可以看到，在系统提示中加入规定格式后，模型可以按照设定的格式回答问题，我们可以方便地通过字符串匹配获得解释或者示例。

> 💡 **提示**：在进行单轮问答时，将内容放在系统提示和用户提示并没有太大的区别，但是在多轮问答时，将不变的规则放入系统提示，变化的规则放入用户提示中会方便许多。

---

### 6.3 清晰指令原则实践

让我们通过对比示例来理解清晰指令的重要性：

In [10]:
system_prompt = "你是一个营养师，擅长为人们制定健康饮食计划。"

# 含糊指令
user_prompt_blur = "怎样吃才能减肥？"

# 清晰指令
user_prompt_clear = "我想减肥，目标是半年内从120斤减到100斤，请为我制定一个包含早餐、午餐和晚餐的健康饮食计划，确保每天摄入的热量不超过1500卡路里。"

answer_blur = get_response(system_prompt, user_prompt_blur)
print("=" * 60)
print("含糊指令回答：")
print("=" * 60)
print(answer_blur)

print("\n" + "=" * 60)
print("清晰指令回答：")
print("=" * 60)
answer_clear = get_response(system_prompt, user_prompt_clear)
print(answer_clear)

含糊指令回答：
减肥的关键在于创造热量赤字，也就是消耗的热量要大于摄入的热量。以下是一些健康饮食的建议，可以帮助你减肥：

1. **控制热量摄入**：
   - 计算每日所需的热量，并确保摄入量低于这个数值。
   - 使用食物记录应用程序来跟踪你的饮食。

2. **选择营养密度高的食物**：
   - 多吃水果、蔬菜、全谷物、瘦肉、鱼类和豆类等富含营养的食物。
   - 避免高糖、高脂肪和高热量的加工食品。

3. **增加蛋白质摄入**：
   - 蛋白质有助于增加饱腹感，减少食欲。可以选择鸡肉、鱼、豆腐、豆类、坚果等。

4. **适量摄入健康脂肪**：
   - 选择橄榄油、牛油果、坚果等健康脂肪，适量摄入有助于维持饱腹感。

5. **多喝水**：
   - 水可以帮助你保持水分，减少饥饿感。餐前喝水也有助于减少食量。

6. **定时进餐**：
   - 规律的饮食时间可以帮助控制饥饿感，避免暴饮暴食。

7. **注意份量控制**：
   - 使用小碗和小盘子来控制食物的份量，避免过量进食。

8. **减少糖分摄入**：
   - 尽量减少含糖饮料和甜点的摄入，选择天然的水果来满足甜食的欲望。

9. **保持活跃**：
   - 结合适量的运动，如步行、跑步、游泳或力量训练，增加热量消耗。

10. **保持良好的睡眠**：
    - 睡眠不足可能会影响荷尔蒙水平，增加食欲，导致体重增加。

记住，减肥是一个长期的过程，保持耐心和坚持是非常重要的。如果有条件，建议咨询专业的营养师，制定个性化的饮食计划。

清晰指令回答：
好的，以下是一个为期一周的健康饮食计划，旨在帮助你在半年内从120斤减到100斤。这个计划每天的热量摄入控制在1500卡路里以内，同时确保营养均衡。

### 一周饮食计划

#### 周一
- **早餐**（约300卡路里）
  - 燕麦粥（50克燕麦，200毫升水） + 1个水煮蛋 + 半个香蕉

- **午餐**（约500卡路里）
  - 煮鸡胸肉（100克） + 西兰花（100克，蒸） + 糙米（50克，煮熟）

- **晚餐**（约400卡路里）
  - 清蒸鱼（100克） + 菠菜（100克，蒸） + 1小碗紫米饭（50克）

- **加餐**（约300卡路里）
  - 1个苹果 + 10颗杏仁

#### 周二

可以看到，当使用含糊指令时，模型回答内容松散，缺乏针对性；而当我们使用清晰的指令阐述需求时，模型便可以根据用户需要做出相应的回答。

---

### 6.4 角色扮演实践

角色扮演是提示工程中非常重要且实用的技巧。让我们通过对比示例来理解角色扮演的效果：

In [11]:
user_prompt = "请简要回答如何治疗感冒"

# 无角色扮演
direct_answer = get_response("", user_prompt)
print("=" * 60)
print("无角色扮演：")
print("=" * 60)
print(direct_answer)

# 有角色扮演
system_prompt = "你是一位经验丰富的医生，擅长治疗各种常见疾病，尤其是感冒。你会非常专业、耐心地回答患者的问题，有效安抚患者情绪，并提供详细的治疗建议。"
response = get_response(system_prompt, user_prompt)
print("\n" + "=" * 60)
print("有角色扮演：")
print("=" * 60)
print(response)

无角色扮演：
治疗感冒的方法主要包括以下几点：

1. **休息**：保证充足的休息，有助于身体恢复。
2. **补充水分**：多喝水、汤或果汁，保持身体水分充足。
3. **对症治疗**：可以使用非处方药物缓解症状，如退烧药、止咳药和抗组胺药。
4. **保持空气湿润**：使用加湿器可以缓解喉咙干燥和鼻塞。
5. **营养饮食**：多吃富含维生素C的水果和蔬菜，增强免疫力。

如果症状严重或持续时间较长，建议咨询医生。

有角色扮演：
治疗感冒通常包括以下几个方面：

1. **休息**：确保充足的休息，帮助身体恢复。

2. **补充水分**：多喝水、汤或果汁，保持身体水分充足。

3. **对症治疗**：
   - **发热和疼痛**：可以使用对乙酰氨基酚或布洛芬等非处方药物来缓解。
   - **鼻塞**：可以使用生理盐水鼻喷剂或去充血剂。
   - **咳嗽**：根据咳嗽类型选择止咳药或化痰药。

4. **保持空气湿润**：使用加湿器可以缓解喉咙和鼻腔的不适。

5. **营养均衡**：多吃富含维生素C的水果和蔬菜，增强免疫力。

如果症状持续超过一周或加重，建议及时就医。希望你早日康复！


总结而言，通过在提示中设定模型角色，能够增强回复专业性，模型会调动该角色相关的内容和表达习惯，输出更专业的内容。

---

### 6.5 思维链（Chain of Thought）实践

思维链技术通过引导模型进行深度思考后再回答，有效提高回复的准确性和逻辑性。让我们通过一个概率问题来对比使用思维链前后的效果：

In [12]:
question = "有甲、乙、丙三个盒子，甲盒中有一个白球和两个黑球，乙盒中有两个白球和一个黑球，丙盒中有三个白球和三个黑球，掷一枚骰子决定盒子。若出现的点数为1，2，3，选择甲盒，若出现的点数为4，选择乙盒，若出现的点数为5，6，选择丙盒。再从选中的盒子中任意选取一球。求：当取出的球为白球时，此球来自甲盒的概率？"

# 无思维链：直接给出答案
system_prompt = "直接给出问题的答案，禁止任何分析和思考。"
response = get_response(system_prompt, question)
print("=" * 60)
print("无思维链（直接回答）：")
print("=" * 60)
print(response)

# 有思维链：逐步思考
system_prompt = "逐步思考并给出问题答案"
response = get_response(system_prompt, question)
print("\n" + "=" * 60)
print("有思维链（逐步思考）：")
print("=" * 60)
print(response)

无思维链（直接回答）：
当取出的球为白球时，此球来自甲盒的概率为 \(\frac{1}{3}\)。

有思维链（逐步思考）：
为了求解这个问题，我们可以使用贝叶斯定理。我们需要计算在取出白球的条件下，球来自甲盒的概率。

首先，我们定义事件：
- \( A_1 \)：从甲盒中取出球
- \( A_2 \)：从乙盒中取出球
- \( A_3 \)：从丙盒中取出球
- \( B \)：取出的球是白球

根据题意，掷骰子的结果决定了选择哪个盒子：
- 选择甲盒的概率 \( P(A_1) = \frac{3}{6} = \frac{1}{2} \)
- 选择乙盒的概率 \( P(A_2) = \frac{2}{6} = \frac{1}{3} \)
- 选择丙盒的概率 \( P(A_3) = \frac{1}{3} \)

接下来，我们计算在每个盒子中取出白球的概率：
- 从甲盒中取出白球的概率 \( P(B | A_1) = \frac{1}{3} \)（甲盒中有1个白球和2个黑球）
- 从乙盒中取出白球的概率 \( P(B | A_2) = \frac{2}{3} \)（乙盒中有2个白球和1个黑球）
- 从丙盒中取出白球的概率 \( P(B | A_3) = \frac{1}{2} \)（丙盒中有3个白球和3个黑球）

接下来，我们使用全概率公式计算取出白球的总概率 \( P(B) \)：
\[
P(B) = P(B | A_1) P(A_1) + P(B | A_2) P(A_2) + P(B | A_3) P(A_3)
\]
代入已知的概率：
\[
P(B) = \left(\frac{1}{3} \cdot \frac{1}{2}\right) + \left(\frac{2}{3} \cdot \frac{1}{3}\right) + \left(\frac{1}{2} \cdot \frac{1}{3}\right)
\]
\[
= \frac{1}{6} + \frac{2}{9} + \frac{1}{6}
\]
为了计算这个和，我们需要找到一个公分母。公分母为18：
\[
= \frac{3}{18} + \frac{4}{18} + \frac{3}{18} = \frac{10}{18} = \frac{5}{9}
\]

现在我们可以使用贝叶斯定理

使用思维链后，模型给出了详细的推理过程，并得出了正确的答案。思维链技术对于复杂推理任务非常有效。

---

### 6.6 上下文学习（In-Context Learning）实践

上下文学习通过在输入中添加适当的示例文本来提高模型能力。以下是一个情感分类的示例：

In [13]:
prompt = """
将文本分类为正面、中性、或负面情感。请根据以下示例进行分类：
文本：我今天过得很开心！
情感：正面
文本：这部电影真是太糟糕了。
情感：负面
文本：我人为这个餐厅的食物还可以。
情感：中性

对以下文本进行分类：
文本：这个地方风景优美，鸟语花香，气候宜人。
情感：
"""
response = get_response("", prompt)
print(response)

情感：正面


通过提供示例，模型能够学习任务模式并按照期望的格式输出结果。

---

### 6.7 提示工程调优完整案例：电影推荐系统

下面我们通过一个完整的电影推荐系统案例，演示如何通过迭代优化提示词来提升模型输出质量。

#### 6.7.1 初版提示词

In [14]:
MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE = "你是一个专业的电影推荐系统，你的任务是根据用户的观影历史，从候选电影列表中推荐最符合用户口味的电影。"

item_lists = """
1. 电影ID: 001, 名称: 《蒙娜丽莎的微笑》
2. 电影ID: 002, 名称: 《肖申克的救赎》
3. 电影ID: 003, 名称: 《阿甘正传》
4. 电影ID: 004, 名称: 《扬名立万》
5. 电影ID: 005, 名称: 《这个杀手不太冷静》
6. 电影ID: 006, 名称: 《美丽人生》
7. 电影ID: 007, 名称: 《电锯惊魂》
8. 电影ID: 008, 名称: 《移动迷宫》
"""

candidates = """
1. 电影ID: 007, 名称: 《盗梦空间》
2. 电影ID: 008, 名称: 《泰坦尼克号》
3. 电影ID: 009, 名称: 《抓娃娃》
4. 电影ID: 010, 名称: 《哪吒之魔童降世》
5. 电影ID: 011, 名称: 《疯狂的石头》
6. 电影ID: 012, 名称: 《神探夏洛克》
7. 电影ID: 013, 名称: 《疯狂动物城》
8. 电影ID: 014, 名称: 《穿普拉达的女王》
"""

MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**回答格式**：你必须严格按照以下要求格式回答

解释: [你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

解释: 用户最近观看的电影包括多种类型，从经典剧情片（如《肖申克的救赎》和《美丽人生》）到轻松幽默的影片（如《这个杀手不太冷静》和《扬名立万》），再到一些悬疑和惊悚片（如《电锯惊魂》）。用户似乎对情感深刻的剧情片和幽默风格的电影有较高的兴趣。基于这些信息，我会优先推荐那些同样具有情感深度或幽默元素的电影。

在候选电影中，《盗梦空间》是一部结合了深刻情感和复杂剧情的科幻片，可能会吸引喜欢《肖申克的救赎》和《美丽人生》的用户。《穿普拉达的女王》则是一部轻松幽默的影片，适合喜欢《扬名立万》和《这个杀手不太冷静》的用户。《疯狂动物城》是一部适合家庭观看的动画片，具有幽默和温暖的情感，可能会吸引用户的兴趣。

推荐: [盗梦空间, 穿普拉达的女王, 疯狂动物城]


在该回答中，模型根据用户看过的电影分析了用户的喜好，但是没有考虑到时间因素。我们针对这一问题改进提示词，进一步明确完成任务时需要关注的内容。

#### 6.7.2 改进版提示词（考虑时间因素）

In [15]:
MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**任务**
你需要根据用户最近的观影记录，通过分析每部电影的风格和内容，以及观看该电影的时间顺序，整合分析用户可能的电影喜好以及近期偏好变化，推荐出用户可能最想观看的下一部电影。输出时，请排序待推荐的候选电影列表，要求按照推荐的优先级排序，推荐程度越高位置越靠前。输出列表中仅包含电影的名称。

**回答格式**：你必须严格按照以下要求格式回答

解释: [你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

解释: 用户最近观看的电影包括多种类型，从经典剧情片（如《肖申克的救赎》和《美丽人生》）到轻松幽默的影片（如《这个杀手不太冷静》和《扬名立万》），再到一些悬疑和惊悚的电影（如《电锯惊魂》）。用户似乎对情感深刻的剧情片和轻松幽默的电影都有兴趣，同时也尝试了一些悬疑和科幻类型的影片。根据这些信息，推荐的电影应当兼顾情感深度和娱乐性。

在候选电影中，《盗梦空间》是一部结合了科幻和心理元素的电影，符合用户对复杂情节的兴趣；《穿普拉达的女王》则是一部轻松幽默的影片，适合用户对轻松内容的偏好；《疯狂动物城》是一部适合家庭观看的动画片，既有趣又富有教育意义，可能会吸引用户的注意。

推荐: [盗梦空间, 穿普拉达的女王, 疯狂动物城]


这一次，模型不仅分析了用户的电影偏好，并且分析出了用户的偏好变化，考虑的信息更加全面，推理结果也更加合理。但是输出格式却出现了问题。这在实际做提示工程时是很容易出现的，因为大模型底层逻辑中的注意力机制，在输入文本过长时，可能会产生忽视关键信息的问题。

#### 6.7.3 最终版提示词（格式优化）

In [16]:
MOVIE_RECOMMEND_PROMPT_CHINESE = """
**输入信息**
1. 用户最近的观影记录，按时间排序，越靠后表示越近期观看，每一项为电影ID和电影名。
{item_list}
2. 待推荐的候选电影列表，每一项为电影ID和电影名。
{candidates}

**任务**
你需要根据用户最近的观影记录，通过分析每部电影的风格和内容，以及观看该电影的时间顺序，整合分析用户可能的电影喜好以及近期偏好变化，推荐出用户可能最想观看的下一部电影。输出时，请排序待推荐的候选电影列表，要求按照推荐的优先级排序，推荐程度越高位置越靠前。输出列表中仅包含电影的名称。

**回答格式**：你必须严格按照以下要求格式回答，否则你的答案会被视为无效。

解释: [详细描述你的思考和分析过程]
推荐: [名称1, 名称2, 名称3, ... 按照推荐程度降序排列电影的名称]
"""

response = get_response(
    MOVIE_RECOMMEND_SYSTEM_PROMPT_CHINESE,
    MOVIE_RECOMMEND_PROMPT_CHINESE.format(
        item_list=item_lists,
        candidates=candidates
    ))    
print(response)

解释: 用户最近观看的电影包括多种类型，从经典剧情片（如《肖申克的救赎》和《美丽人生》）到轻松幽默的影片（如《扬名立万》和《这个杀手不太冷静》），再到一些悬疑和惊悚元素的电影（如《电锯惊魂》）。用户的观影记录显示出对情感深刻、剧情引人入胜的电影的偏好，同时也对幽默和轻松的内容有一定的兴趣。

在候选电影中，《盗梦空间》是一部结合了科幻和心理惊悚的电影，符合用户对复杂剧情的偏好；《穿普拉达的女王》则是一部轻松幽默的影片，可能吸引用户对轻松内容的兴趣；《疯狂动物城》是一部适合家庭观看的动画片，具有幽默和温暖的情感，可能会吸引用户的注意。

综合考虑用户的观影历史和候选电影的风格，推荐的优先级如下：

推荐: [盗梦空间, 穿普拉达的女王, 疯狂动物城]


通过强调格式要求的重要性（"否则你的答案会被视为无效"），模型现在能够更好地遵循输出格式。

> 💡 **总结**：提示调优过程为：初版提示 → 观察模型表现，找出问题点 → 针对性改写提示，逐轮试验调整 → 直至输出符合预期。提示工程是一个实验性很强、非常依赖试错的过程，没有"万能好用"的初版提示，调优迭代是提示工程中最核心、最真实的工作方式。

---

## 7. 代码要点总结

### 7.1 提示词设计要点
- ✅ **系统提示**：设定角色、行为准则和输出格式
- ✅ **用户提示**：包含具体的任务指令和上下文信息
- ✅ **清晰具体**：避免模糊表达，提供详细指令
- ✅ **格式约束**：明确输出格式要求，便于后续处理

### 7.2 常用技术实践
- ✅ **角色扮演**：通过设定角色提升专业性
- ✅ **思维链**：引导模型逐步推理，提高准确性
- ✅ **上下文学习**：通过示例引导模型学习任务模式

### 7.3 调优流程
- ✅ **观察输出偏差**：分析模型输出与预期目标的差距
- ✅ **迭代优化**：通过反复试验不断调整提示词
- ✅ **格式强化**：在关键位置强调格式要求

---

## 8. 注意事项

1. **API 配置**：确保 HandsOn/config.json 文件已正确配置，包含所需的 API 密钥和平台信息
2. **提示词长度**：注意提示词长度限制，避免超出模型上下文窗口
3. **格式要求**：在提示词中明确格式要求，并在关键位置强调
4. **迭代优化**：提示工程需要反复试验和优化，要有耐心
5. **成本控制**：注意 API 调用成本，合理设置 max_tokens 等参数